In [ ]:
from data.get_tickers import get_tickers
from data.get_stock_data import get_ohlcv_batch
from mean import LLTrendBatch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
tickers = get_tickers().query("Market == 'Oslo Børs'").Symbol.to_list()
ohlcv_batch = get_ohlcv_batch(tickers, interval ="1h", update=False).drop_illiquid(500_000)
lltrend_batch = LLTrendBatch.fit(batch=ohlcv_batch)

In [ ]:
lltrend_batch.er().sort_values("er", ascending=False)

In [ ]:
model = lltrend_batch.items["KIT.OL"]
fig, ax = plot_lltrend_forecast(model, days=30, z=1.65, alpha=0.10, tail=2000)
plt.show()

In [ ]:
from typing import Optional

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


def _forecast_steps(interval: str, days: int = 7) -> int:
    if interval == "1h":
        return 8 * days
    if interval == "1d":
        return days
    raise ValueError(f"Unsupported interval: {interval!r} (supported: '1h', '1d')")


def _forecast_index(idx: pd.Index, steps: int) -> pd.Index:
    if isinstance(idx, pd.DatetimeIndex):
        freq = idx.freq or idx.inferred_freq
        if freq is None:
            # fallback: hourly/daily guessing from last two stamps, else just RangeIndex
            if len(idx) >= 2:
                delta = idx[-1] - idx[-2]
                freq = "h" if delta <= pd.Timedelta(hours=2) else "d"
            else:
                return pd.RangeIndex(steps)
        start = idx[-1] + pd.tseries.frequencies.to_offset(freq)
        return pd.date_range(start=start, periods=steps, freq=freq, name=idx.name)
    return pd.RangeIndex(steps)


def plot_lltrend_forecast(
    m: "LLTrendModel",
    *,
    days: int = 7,
    z: float = 2.0,
    alpha: float = 0.05,
    lags: int = 24,
    tail: int | None = None,
    ax: Optional[list] = None,
):
    """
    Two panels:
      1) Close + filtered level, plus next-week forecast (mean + interval)
      2) Filtered trend (mean + z band), plus forecasted trend (mean + z band) and y=1 dotted
    """
    steps = _forecast_steps(m.ohlcv.interval, days=days)

    # In-sample (filtered) states (unlogged)
    states = m.unlogged(z=z)
    if tail is not None:
        states = states.iloc[-tail:]

    # Align close to model index
    close = pd.to_numeric(m.ohlcv.df.get("Close"), errors="coerce").reindex(states.index)

    # Forecast of endog (log-close), convert to price
    pred = m.res.get_forecast(steps=steps)
    f_idx = _forecast_index(states.index, steps)

    mean = np.asarray(getattr(pred, "predicted_mean"))
    if mean.ndim > 1:
        mean = mean.ravel()
    mean_s = pd.Series(mean, index=f_idx)

    ci = pred.conf_int(alpha=alpha)
    ci_arr = np.asarray(ci)
    if ci_arr.ndim == 3:
        ci_arr = ci_arr[:, 0, :]  # (steps, k_endog, 2) -> (steps, 2) for univariate

    lo_s = pd.Series(ci_arr[:, 0], index=f_idx)
    hi_s = pd.Series(ci_arr[:, 1], index=f_idx)

    f_close = mean_s.pipe(np.exp)
    f_close_lo = lo_s.pipe(np.exp)
    f_close_hi = hi_s.pipe(np.exp)

    # Try to get forecasted states (level/trend) for panel 2
    ps = getattr(pred, "predicted_state", None)
    psc = getattr(pred, "predicted_state_cov", None)
    if ps is None or psc is None:
        pr = getattr(pred, "prediction_results", None)
        ps = getattr(pr, "predicted_state", None) if pr is not None else None
        psc = getattr(pr, "predicted_state_cov", None) if pr is not None else None

    f_tr = f_tr_lo = f_tr_hi = None
    if ps is not None and psc is not None:
        ps = np.asarray(ps)          # (k_states, steps)
        psc = np.asarray(psc)        # (k_states, k_states, steps)

        tr_log = pd.Series(ps[1, :], index=f_idx)  # state 1 = trend for local linear trend
        tr_se = pd.Series(np.sqrt(psc[1, 1, :]), index=f_idx)

        f_tr = tr_log.pipe(np.exp)
        f_tr_lo = (tr_log - z * tr_se).pipe(np.exp)
        f_tr_hi = (tr_log + z * tr_se).pipe(np.exp)

    # --- plotting ---
    if ax is None:
        fig, ax = plt.subplots(2, 1, sharex=True, figsize=(12, 7))
    else:
        fig = ax[0].figure

    # Panel 1: data vs forecast (price)
    ax[0].plot(close.index, close.values, label="Close (data)")
    ax[0].plot(states.index, states["level"].values, label="Level (filtered)")

    ax[0].plot(f_close.index, f_close.values, linestyle="--", label=f"Close forecast ({days}d)")
    ax[0].fill_between(
        f_close.index,
        f_close_lo.to_numpy(),
        f_close_hi.to_numpy(),
        alpha=0.2,
        label=f"{int((1 - alpha) * 100)}% forecast interval",
    )

    ax[0].set_title(f"{m.ohlcv.ticker} [{m.ohlcv.interval}] | p_white(min, lags={lags})={m._p_white(lags=lags):.3g}")
    ax[0].legend()

    # Panel 2: trend (data + forecast if available)
    ax[1].plot(states.index, states["trend"].values, label="Trend (filtered)")
    ax[1].fill_between(
        states.index,
        states["trend_lo"].to_numpy(),
        states["trend_hi"].to_numpy(),
        alpha=0.2,
        label=f"{z}σ band (filtered)",
    )

    if f_tr is not None:
        ax[1].plot(f_tr.index, f_tr.values, linestyle="--", label="Trend forecast")
        ax[1].fill_between(
            f_tr.index,
            f_tr_lo.to_numpy(),
            f_tr_hi.to_numpy(),
            alpha=0.2,
            label=f"{z}σ band (forecast)",
        )

    ax[1].axhline(1.0, linestyle=":", linewidth=1, label="1.0")
    ax[1].legend()

    fig.tight_layout()
    return fig, ax

In [ ]:
lltrend_batch.items["EQNR.OL"].er()

In [ ]:
df = get_ohlcv(csv_path='data/oslo_stock_exchange_ohlcv.csv', long=True)
df["log_return"] = df.groupby("Ticker")["Close"].transform(lambda s: np.log(s).diff())
df["log_close"] = np.log(df["Close"])
stock = df.query("Ticker == 'DNO.OL'").copy()
stock = stock.iloc[0:5200].copy().reset_index()

In [ ]:
pd.set_option("display.float_format", lambda x: f"{x:.8g}")
df['monetary_volume'] = df['Close'] * df['Volume']
 = df.groupby("Ticker")["monetary_volume"].median().sort_values()
volumes[volumes > 1_000_000]

In [ ]:
df